# 4. Returning function

As functions are objects, we can also return a function from another function. In the below example, the create_adder function returns adder function.

## 4.1 Simple example
In below example, the counter() function returns another function count

In [1]:
def counter(step: int):
    def count(start: int):
        return start + step

    return count

You can check the type of c1

In [2]:
c1 = counter(step=2)
print(type(c1))

<class 'function'>


In [3]:
res = c1(start=6)
print(type(res))
print(res)

8

In fact, the returned function count() is a **nested function** inside counter(). You can also notice that the nested function **can access variables of the enclosing scope**


## 4.2 Nonlocal variable in a nested function

In previous example, we have seen that **a function defined inside another function (nested function). Nested functions can access variables of the enclosing scope**.

In Python, **these non-local variables are read-only by default and we must declare them explicitly as non-local (using nonlocal keyword) in order to modify them**.

Consider below example

In [ ]:
def func_1():
    x = "value_defined_by_func_1"

    # define function
    def func_2():
        x = "value_defined_by_func_2"
        return x

    # run the function
    func_2()
    return func_2, x

In [10]:
f2, x = func_1()

print(type(f2))
print(type(x))

print(f2())
print(x)

In above example, you can notice that the func_2 does not change the value of x in func_1. This is normal due to the scope of x. But what if I want func_2 change the
value of x in func_1. We need to **use the keyword nonlocal**.

Check below example.

In [ ]:
def func_1():
    x = "value_defined_by_func_1"

    # define a function
    def func_2():
        nonlocal x
        x = "value_defined_by_func_2"
        return x

    # run the function
    func_2()
    return func_2, x

In [12]:
f2, x = func_1()

print(f2())
print(x)

This time the value of x in func_1 has been changed by func_2. This because we declare x as nonlocal in func_2.

## 4.3 Closure

In above examples, we **attached data to a nested function, then we return the nested function. This technique is called closure in Python.** This **value in the enclosing scope is remembered even when the variable goes out of scope or the function itself is removed from the current namespace**.

Try running the following in the Python shell to see the output.

In [14]:
def store_data(data):
    def disk():
        print(data)

    def memory():
        print(data)
    return disk, memory

In [15]:
d1,m1=store_data("this is my secret")
d1()
m1()

this is my secret
this is my secret


So far, nothing special. Let's delete the parent function store_data

In [16]:
del store_data

In [17]:
store_data("I'm not here")

NameError: name 'store_data' is not defined

In [18]:
d1()
m1()

this is my secret
this is my secret


You can noticed, even the parent get deleted, the returned nested function still works and data is not lost. That's because after the parent return the two nested function. Two new function object has been created with the value of data at the moment of their creation. After that, the two objects are independent.

## 4.4 Values of the parent variable

And we must be extremely careful when we use parent variables in nested function. check below example, and check the return value. Do you expect the result would be [1,4,9]?

In [ ]:
def count():
    func_list = []
    for i in range(1, 4):
        def f():
            return i * i

        func_list.append(f)
    return func_list


f1, f2, f3 = count()

print(f1())
print(f2())
print(f3())

The result is 9,9,9. That's because the nested function are not been executed when they are returned. A function object only gets created and takes the value of i when they are executed. In our case, when they are executed, the loop ends and the value of i is 3.

**Rule1: Never use a variable in a nested function whose value changes in an outer loop**

If you have to have a loop,

In [19]:
def count():
    def f(j):
        def g():
            return j*j
        return g
    func_list = []
    for i in range(1, 4):
        func_list.append(f(i)) # f(i)gets executed, the function object is created with current i value
    return func_list

In [20]:
f1,f2,f3=count()
print(f1())
print(f2())
print(f3())

1
4
9


## 4.5 When do we have closures?
As seen from the above example, we have a closure in Python when a nested function references a value in its enclosing scope.

The criteria that must be met to create closure in Python are:.

- We must have a nested function (function inside a function).
- The nested function must refer to a value defined in the enclosing function.
- The enclosing function must return the nested function.

## 4.6 When to use closures?
So what are closures good for?

**Closures can avoid the use of global values and provides some form of data hiding.** It can also provide an object oriented solution to the problem.

When there are few methods (one method in most cases) to be implemented in a class, closures can provide an alternate and more elegant solution. But when the number of attributes and methods get larger, it's better to implement a class.

**Python Decorators** make an extensive use of closures as well.

Here is a simple example where a closure might be more preferable than defining a class and making objects. But the preference is all yours.

In [22]:
def make_multiplier_of(n):
    def multiply(x):
        return x*n
    return multiply

time3=make_multiplier_of(3)
time5=make_multiplier_of(5)

print(time3(2))
print(time3(3))

print(time5(2))
print(time5(3))

6
9
10
15
